# FT用データ生成スクリプト

In [1]:
# テキスト生成AIに関係するパッケージインストール
# -U: 最新版のインストール
!pip install -U openai \
    langchain==0.3.12 \
    langchain-community \
    langchain-core \
    langchain-google-genai \
    langchain-openai \
    langgraph \
    python-dotenv

# rag用パッケージのインストール
!pip install -U chromadb \
    langchain-chroma \
    pypdf \
    pdfminer.six

# coquiインストール
!pip install TTS[ja]

# 文字起こし用パッケージインストール
!pip install git+https://github.com/openai/whisper.git

!pip install librosa

  Using cached openai-2.1.0-py3-none-any.whl.metadata (29 kB)
  Using cached langchain_community-0.3.30-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_google_genai-2.1.12-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_openai-0.3.34-py3-none-any.whl.metadata (2.4 kB)
  Using cached langgraph-0.6.8-py3-none-any.whl.metadata (6.8 kB)
  Using cached langsmith-0.2.11-py3-none-any.whl.metadata (14 kB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.3.78-py3-none-any.whl.metadata (3.2 kB)
  Using cached langchain_core-0.3.77-py3-none-any.whl.metadata (3.2 kB)
  Using cached langchain_core-0.3.76-py3-none-any.whl.metadata (3.7 kB)
  Using cached langchain_core-0.3.75-py3-none-any.whl.metadata (5.7 kB)
INFO: pip is still looking at mu

['Collecting git+https://github.com/openai/whisper.git',
 '  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-9t9rxrkw',
 '  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-9t9rxrkw',
 '  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2',
 '  Installing build dependencies: started',
 "  Installing build dependencies: finished with status 'done'",
 '  Getting requirements to build wheel: started',
 "  Getting requirements to build wheel: finished with status 'done'",
 '  Preparing metadata (pyproject.toml): started',
 "  Preparing metadata (pyproject.toml): finished with status 'done'",
 'Requirement already satisfied: more-itertools in /home1/s1f102201582/anaconda3/envs/py10_jupy_only/lib/python3.10/site-packages (from openai-whisper==20250625) (10.7.0)',
 'Requirement already satisfied: numba in /home1/s1f102201582/anaconda3/envs/py10_jupy_only/lib/python3.

## テキスト対話データ生成

In [2]:
import os
from typing import Literal
import ast

from dotenv import load_dotenv
from langchain import hub
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PDFMinerLoader, PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI


# .envファイル読み込み
load_dotenv()

/home1/s1f102201582/anaconda3/envs/py10_jupy_only/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
#config
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
BASE_URL = "https://api.openai.iniad.org/api/v1"
MODEL='gemini-2.5-flash'
TEMPERATURE = 2.0
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

# 生成する音声のサンプリングレート
sr = 16000

#対話音声データの個数を指定
gen_dial_num = 1

# すでに作成した対話データを削除するかどうか
IS_REMOVE_EXIST_FILE = False

# ftに使うjsonとaudioの出力フォルダパス
home_dir = expanduser("~")
json_dir_path = join(home_dir, "Github/jmoshi-ft/gen_dialogue/data/transcription")
audio_dir_path = join(home_dir, "Github/jmoshi-ft/gen_dialogue/data/audio")

In [ ]:
exist_check_paths = [
    json_dir_path,
    audio_dir_path,
]

for p in exist_check_paths:
    if not os.path.isdir(p):
        os.makedirs(p)

In [4]:
# client作成
llm = ChatGoogleGenerativeAI(
                 model=MODEL,
                 temperature=TEMPERATURE)

In [5]:
loader = DirectoryLoader(
    "../../mental_docs/",
    glob="*.pdf",
    show_progress=True,
    loader_cls=PyPDFLoader,
    # loader_cls=PDFMinerLoader
)
docs = loader.load()
print(f"Loaded {len(docs)} documents")

100%|███████████████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.25it/s]

Loaded 159 documents


In [6]:
# Debug
# for doc in docs:
#     print("-------------------------------------------------")
#     print(doc.metadata)
#     print(len(doc.page_content))
#     print(doc.page_content[:100])

In [7]:
#読み込んだ文章データをオーバーラップ200文字で1000文字づつ分割
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)

# 埋め込み
embedding = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=BASE_URL,
    model="text-embedding-3-small"
)

#ベクトルデータベースのChromaDBaに保存
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embedding
)

In [8]:
# vectorstoreから必要な情報を読み出す
retriever = vectorstore.as_retriever()

# ユーザーが与えるプロンプトに加えて、
# rag_promptを追加してLLMに与えるように設定
rag_prompt = hub.pull("rlm/rag-prompt")

In [9]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [10]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [11]:
#promptを作成
prompt_txt = """臨床心理士が行うメンタルヘルスケアカウンセリングをシミュレーションし、その対話内容に相槌を含め、話し言葉のまま文字起こししてください。
会話は中途半端で終わらせず、きりが良い会話にしてください。
相槌は実際の対話を想定して細かく入れてください。
語感は固くならないようにしてください。

形式は以下のようにしてください。Aがカウンセラーで、Bがカウンセリングを受ける人です。
以下の例は３回しか言葉を交わしていませんが、500文字程度の会話になるようにしてください。
カウンセリングで話す悩みは仕事以外にも考えうる様々なテーマすべて取り扱ってください。
文字列内に「A: 」のような誰が話したのかを明記する必要はありません。

[
 "Aが話す言葉",
 "Bが話す言葉",
 "Aが話す言葉",
 ...
]
"""

In [12]:
# テキスト対話生成関数
def gen_txt_dialogue():
    return rag_chain.invoke(prompt_txt)

In [13]:
# 対話テキストから対話テキストリスト生成関数
def txt_to_lst(dialogue_txt):
    return ast.literal_eval(dialogue_txt)

In [14]:
# DEBUG
# txt_dialogue = gen_txt_dialogue()
# print(txt_dialogue)
# lst_dialogue = txt_to_lst(txt_dialogue)
# print(lst_dialogue)

## テキスト対話データを音声対話データに変換 

In [15]:
from collections import defaultdict

import torch
import librosa
import numpy as np
import soundfile as sf
from TTS.utils.radam import RAdam
from TTS.api import TTS

model_name = "tts_models/ja/kokoro/tacotron2-DDC"

# デバイス設定
device = "cuda" if torch.cuda.is_available() else "cpu"

# PyTorchのtorch.loadのweights_only引数の仕様変更によるエラー防止
# 許可リストに追加 (これをしないとエラーが出る)
torch.serialization.add_safe_globals([RAdam])
torch.serialization.add_safe_globals([defaultdict])
torch.serialization.add_safe_globals([dict])

# モデルをダウンロードしていない場合は例外が出る
while True:
    try:
        model_path = os.path.expanduser("~/.local/share/tts/tts_models--ja--kokoro--tacotron2-DDC/model_file.pth")

        # 安全にロード（weights_only=TrueにしてもOK）
        model1 = torch.load(model_path, weights_only=True)
        break
        
    except Exception:
        # モデルをダウンロード
        TTS(model_name=model_name)


In [16]:
# モデル名指定してTTSインスタンス作成
tts = TTS(model_name=model_name).to(device)

 > tts_models/ja/kokoro/tacotron2-DDC is already downloaded.
 > vocoder_models/ja/kokoro/hifigan_v1 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > pitch_fmin:0.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:/users/s1f102201582/.local/share/tts/tts_models--ja--kokoro--tacotron2-DDC/scale_stats.npy
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model's reductio

In [17]:
def tts_coqui(text: str):
    wav = tts.tts(text)
    return wav

In [18]:
def lst_to_audio_dialogue(lst_dialogue):
    # 音声ファイルを順番に生成（ファイルは不要なのでwave配列で持つ）
    wav_data = []
    for i, text in enumerate(lst_dialogue):
        speaker = "A" if i%2==0 else "B"
        wav = tts_coqui(text)
        
        # numpy配列でなければ変換（torch.Tensorやlistの場合にも対応）
        if not isinstance(wav, np.ndarray):
            wav = np.array(wav, dtype=np.float32)
        
        if sr != tts.synthesizer.output_sample_rate:
            wav = librosa.resample(wav, orig_sr=tts.synthesizer.output_sample_rate, target_sr=sr)
        wav_data.append(wav)
    
    # 最終的な音声長を決定
    max_len = sum([len(w) for w in wav_data])
    
    # ステレオ音声用（2チャンネル×最大長）の空配列をゼロ初期化で作成
    stereo = np.zeros((2, max_len), dtype=np.float32)
    
    pos = 0
    for i, wav in enumerate(wav_data):
        ch = i%2  # 0:左(A), 1:右(B)
        stereo[ch, pos:pos+len(wav)] += wav
        pos += len(wav)
    
    # 転置し(-1,2)する
    stereo = stereo.T
    return stereo

## 音声対話データを文字起こししたjsonファイルを生成

In [19]:
import whisper
import torch
import soundfile as sf
import numpy as np

device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')

# Whisperモデルをロード
model = whisper.load_model("large-v3-turbo", device=devide)

In [20]:
def transcribe_channel(channel, sr, speaker_label):
    # Whisper expects mono wav, so pass as-is
    result = model.transcribe(
        channel.astype(np.float32), 
        language='ja', 
        word_timestamps=True  # NOTE: requires Whisper >=2023.4
    )
    # word-level JSON extraction
    words_json = []
    for segment in result['segments']:
        for word in segment['words']:   # word-level timestamps
            words_json.append({
                'speaker': speaker_label,
                'word': word['word'],
                'start': word['start'],
                'end': word['end']
            })
    return words_json

def transcribe_audio_dialogue(audio_path):
    # ステレオ分離: speaker A=左(0), B=右(1)と仮定
    audio, sr = sf.read(audio_path)    # (samples, channels)
    channel_A = audio[:,0]
    channel_B = audio[:,1]
    # 両チャンネルを transcribe
    json_A = transcribe_channel(channel_A, sr, "A")
    json_B = transcribe_channel(channel_B, sr, "B")
    
    # 発話時間でソート（複数話者の時系列並び用）
    full_json = json_A + json_B
    full_json_sorted = sorted(full_json, key=lambda x: x['start'])
    return full_json_sorted

In [21]:
import re

def get_continuous_num_on_wav_file():
    wav_file_pattern = r"^(\d+)\.wav$"
    max_num = -1
    for file in os.listdir(audio_dir_path):
        if not os.path.exists(os.path.join(audio_dir_path, file)):
            continue
        if not re.match(wav_file_pattern, file):
            continue
    
        match_obj = re.match(wav_file_pattern, file)
        get_number = int(match_obj.groups()[0])
    
        if max_num < get_number:
            max_num = get_number
    return max_num

In [22]:
from glob import glob

if IS_REMOVE_EXIST_FILE:
    max_num = -1

    # jsonファイルがある場合、削除
    json_file_paths = glob(os.path.join(json_dir_path, "*"))
    for file_path in json_file_paths:
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"{file_path} を削除しました。")

    # 音声ファイルがある場合、削除
    audio_file_paths = glob(os.path.join(audio_dir_path, "*"))
    for file_path in audio_file_paths:
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"{file_path} を削除しました。")
    
else:
    max_num = get_continuous_num_on_wav_file()

../data/transcription/12.json を削除しました。
../data/transcription/13.json を削除しました。
../data/transcription/17.json を削除しました。
../data/transcription/5.json を削除しました。
../data/transcription/19.json を削除しました。
../data/transcription/4.json を削除しました。
../data/transcription/9.json を削除しました。
../data/transcription/1.json を削除しました。
../data/transcription/10.json を削除しました。
../data/transcription/7.json を削除しました。
../data/transcription/14.json を削除しました。
../data/transcription/3.json を削除しました。
../data/transcription/2.json を削除しました。
../data/transcription/6.json を削除しました。
../data/transcription/11.json を削除しました。
../data/transcription/15.json を削除しました。
../data/transcription/8.json を削除しました。
../data/transcription/16.json を削除しました。
../data/transcription/0.json を削除しました。
../data/transcription/18.json を削除しました。
../data/audio/13.wav を削除しました。
../data/audio/1.wav を削除しました。
../data/audio/2.wav を削除しました。
../data/audio/6.wav を削除しました。
../data/audio/0.wav を削除しました。
../data/audio/9.wav を削除しました。
../data/audio/14.wav を削除しました。
../data/audio/5.wav を削除しま

In [23]:
for i in range(max_num+1, gen_dial_num+max_num+1):

    # 生成AIがリストのフォーマットでテキストを出力できない場合もあるので例外処理
    while True:
        try:
            txt_dialogue = gen_txt_dialogue()
            lst_dialogue = txt_to_lst(txt_dialogue)
            break
        except SyntaxError:
            pass
        
    stereo = lst_to_audio_dialogue(lst_dialogue)

    wav_name = f"{i}.wav"
    audio_file_path = os.path.join(audio_dir_path, wav_name)
    sf.write(audio_file_path, stereo, sr)

    json_data = transcribe_audio_dialogue(audio_file_path)

    json_name = f"{i}.json"
    json_file_path = os.path.join(json_dir_path, json_name)
    
    # JSON出力
    with open(json_file_path, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=2)

 > Text splitted to sentences.
['こんにちは。', '今日はどうぞよろしくお願いしますね。']
 > Processing time: 0.5852231979370117
 > Real-time factor: 0.1326061689669425
 > Text splitted to sentences.
['はい、お願いします。', 'あの、最近、家族との関係で、ちょっと落ち込み気味で…夜もなかなか寝付けない日が続いてて。']
 > Processing time: 0.37731051445007324
 > Real-time factor: 0.03845444850809845
 > Text splitted to sentences.
['うんうん、そうなんですね。', 'ご家族の関係で、落ち込んでいらっしゃるんですね。', 'それはお辛いですね。', '何か具体的な出来事がおありでしたか？']
 > Processing time: 0.5055043697357178
 > Real-time factor: 0.04405819691007058
 > Text splitted to sentences.
['はい。', '先日、些細なことで母と口論になってしまって。', '私としては、色々と気を遣って頑張っているのに、全然認めてもらえないなって感じて…']
 > Processing time: 0.5162017345428467
 > Real-time factor: 0.041623691733477304
 > Text splitted to sentences.
['なるほど。', 'うんうん、ご自身の頑張りがお母様に認めてもらえないと感じて、つらいお気持ちなんですね。', 'はい、その寂しいお気持ち、とてもよく分かります。']
 > Processing time: 0.48639869689941406
 > Real-time factor: 0.03773676767238107
 > Text splitted to sentences.
['はい…。', 'すごく、寂しくて。', '誰にも話せず抱え込んでたから、本当にしんどかったんだなって、今話して気づきました。']
 > P